In [25]:
import pandas
from snorkel.utils import probs_to_preds
from utils import load_spam_dataset
from wrench.dataset import load_dataset
from wrench.endmodel import EndClassifierModel
from wrench.labelmodel import MajorityVoting, Fable

path_to_data = "/Users/asedova/PycharmProjects/01_datasets/wrench/"
# os.chdir("wrench/spam")

In [2]:
!which python

/Users/asedova/PycharmProjects/WS_tutorial/venv/bin/./python


# Weak Supervision

todo: some intro, what is WS once again, what do we need to train a classifier

todo: intro to the dataset, LFs, output

YouTube comments dataset:
- HAM: comments relevant to the video (even very simple ones), or
- SPAM: irrelevant (often trying to advertise something) or inappropriate messages

In [3]:
# load the YouTube dataset
df_train, df_test = load_spam_dataset(load_train_labels=True)
Y_train = df_train["label"].values
Y_test = df_test["label"].values

,author,date,text,label,video
0,Alessandro leite,2014-11-05T22:21:36,pls http://www10.vakinha.com.br/VaquinhaE.aspx...,1,1
1,Salim Tayara,2014-11-02T14:33:30,"if your like drones, plz subscribe to Kamal Ta...",1,1
2,Phuc Ly,2014-01-20T15:27:47,go here to check the views :3﻿,0,1
3,DropShotSk8r,2014-01-19T04:27:18,"Came here to check the views, goodbye.﻿",0,1
4,css403,2014-11-07T14:25:48,"i am 2,126,492,636 viewer :D﻿",0,1
5,Giang Nguyen,2014-11-06T04:55:41,https://www.facebook.com/teeLaLaLa﻿,1,1
6,Caius Ballad,2014-11-13T00:58:20,imagine if this guy put adsense on with all th...,0,1
7,Holly,2014-11-06T13:41:30,Follow me on Twitter @mscalifornia95﻿,1,1
8,King uzzy,2014-11-07T23:19:08,Can we reach 3 billion views by December 2014? ﻿,0,1
9,iKap Taz,2014-11-08T13:34:27,Follow 4 Follow @ Va...,1,1


In [19]:
df_train[:10]

SyntaxError: invalid syntax (2550162073.py, line 1)

Examples of SPAM messages (label = 1):
- "Please check out my vidios"
- "Subscribe to me and I'll subscribe back!!!"

Examples of HAM messages (label = 0):
- "3:46 so cute!"
- "This is a weird video."

todo: what do we have here? text + gold labels. What if there are no gold labels?

What can be a labeling function?

- Keyword searches: looking for specific words in a sentence
- Pattern matching: looking for specific syntactical patterns
- Third-party models: using an pre-trained model (usually a model for a different task than the one at hand)
...
- Crowdworker labels: treating each crowdworker as a black-box function that assigns labels to subsets of the data

For detection of YouTube comments dataset: e.g. "check out" - a marker of a SPAM comment

In [ ]:
# an example of LF based on a key word "check out"
def check_out(x):
    return 1 if "check out" in x.text.lower() else -1

# an example of LF based on a key word "check"
def check(x):
    return 1 if "check" in x.text.lower() else -1

todo: e.g., we created 10 LFs and applied them for each sentence. The result can be saved in the following format:

In [13]:
with open("/Users/asedova/PycharmProjects/01_datasets/wrench/youtube/train.json") as train_file:
    train_data = pandas.read_json(train_file)
print(train_data)

,data,label,weak_labels
0,{'text': 'pls http://www10.vakinha.com.br/Vaqu...,1,"[-1, -1, 1, -1, -1, -1, -1, -1, -1, -1]"
1,"{'text': 'if your like drones, plz subscribe t...",1,"[-1, 1, -1, 1, -1, -1, -1, -1, -1, 0]"
2,{'text': 'go here to check the views :3﻿'},0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"
3,"{'text': 'Came here to check the views, goodby...",0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"
4,"{'text': 'i am 2,126,492,636 viewer :D﻿'}",0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"


todo: explain what are the data, labels (=still gold ones), weak_labels (=the annotations after applying LFs)

todo: what is majority vote?
todo: how to get the weak labels from the wrench data to train a classifier?
todo: example of classifier training
todo: other labeling model (e.g. Snorkel)

## How to obtain weak labels?

todo: different labeling functions: 1) (simple) majority vote 2) (more advanced) FABLE

In [ ]:
train_data, valid_data, test_data = load_dataset(
    path_to_data,
    "youtube",
    extract_feature=True,
    extract_fn='tfidf'
)

1. Majority Vote
(+ todo: a small description)

In [7]:
# initialize and apply the majority vote
label_model = MajorityVoting()
label_model.fit(dataset_train=train_data, dataset_valid=valid_data)

In [34]:
# calculate labels
soft_label_mv = label_model.predict_proba(train_data)
hard_label_mv = probs_to_preds(soft_label_mv)

  0%|          | 0/1586 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

2. Fable
(+ todo: a small description)

In [36]:
# initialize and apply the fable model
label_model = Fable(kernel_function=None, num_groups=3)
label_model.fit(dataset_train=train_data, dataset_valid=valid_data)

In [37]:
# calculate labels
soft_label_fable = label_model.predict_proba(train_data)
hard_label_fable = probs_to_preds(soft_label_mv)

array([1, 1, 0, ..., 1, 1, 1])

## How to train a classifier?

In [30]:
batch_size = 32
test_batch_size = 32
lr = 0.01

model = EndClassifierModel(
    batch_size=batch_size, test_batch_size=test_batch_size
)

AttributeError: 'DataFrame' object has no attribute 'weak_labels'